**Data Preprocessing**:

Normalization: Scale continuous features to a similar range to help the CNN model train more efficiently.

Reshape Data: Although CNNs are typically used for spatial data (e.g., images), we can reshape our time-series data into a 2D format that a CNN can process. Each row (patient's record) can be considered a "channel", similar to how images have RGB channels.

**Feature Selection**:
Given the wide range of features, including lab test results and categorical data, we will initially use all available features. Based on the model's performance, we can later perform feature importance analysis to refine our selection.

**Model Architecture**:
Input Layer: Shape the input layer to match the dimensions of our preprocessed data.

Convolutional Layers: These will learn spatial hierarchies from the data. We can experiment with different numbers of filters and kernel sizes.

Pooling Layers: Use pooling to reduce the dimensions of the data progressively.

Flatten Layer: Flatten the output from the convolutional and pooling layers to a 1D vector for the final prediction.

Dense Layers: After flattening, use one or more dense layers for prediction.
Output Layer: Since we are predicting a continuous value (time from admission to ICU out time), the output layer should have a single neuron.

**Compilation and Training**:
Loss Function: Use a regression loss function, such as Mean Squared Error (MSE), since this is a regression problem.

Optimizer: An optimizer like Adam is typically a good starting point.

Metrics: Track metrics like Mean Absolute Error (MAE) to monitor performance during training.

**Evaluation and Refinement**:
Depending on the initial results, refine the model by adjusting its architecture, tuning hyperparameters, or revisiting the feature selection process.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
train_data = pd.read_excel('/content/drive/MyDrive/SPH 6004/Assignment 2/static_train_df.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/SPH 6004/Assignment 2/static_test_df.xlsx')

Mounted at /content/drive


In [ ]:
los_icu_stats = train_data['los_icu'].describe()

los_icu_stats

count    14289.000000
mean         4.861710
std          6.045013
min          1.000000
25%          1.800000
50%          2.880000
75%          5.260000
max        101.730000
Name: los_icu, dtype: float64

In [ ]:
# Ensure datetime columns are correctly formatted
train_data['hosp_admittime'] = pd.to_datetime(train_data['hosp_admittime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
train_data['hosp_dischtime'] = pd.to_datetime(train_data['hosp_dischtime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
train_data['icu_intime'] = pd.to_datetime(train_data['icu_intime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
train_data['icu_outtime'] = pd.to_datetime(train_data['icu_outtime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
# train_data['los_icu'] = pd.to_datetime(train_data['los_icu'])

# Calculate target variable in training data
train_data['time_to_icu_discharge'] = train_data['icu_outtime'] - train_data['hosp_admittime']

test_data['hosp_admittime'] = pd.to_datetime(test_data['hosp_admittime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
test_data['hosp_dischtime'] = pd.to_datetime(test_data['hosp_dischtime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
test_data['icu_intime'] = pd.to_datetime(test_data['icu_intime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
test_data['icu_outtime'] = pd.to_datetime(test_data['icu_outtime'], format='%m/%d/%y %H:%M').astype('int64') // 10**9
# test_data['los_icu'] = pd.to_datetime(test_data['los_icu'])

test_data['time_to_icu_discharge'] = test_data['icu_outtime'] - train_data['hosp_admittime']

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14289 entries, 0 to 14288
Data columns (total 48 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id                                                               14289 non-null  int64  
 1   hosp_admittime                                                   14289 non-null  int64  
 2   hosp_dischtime                                                   14289 non-null  int64  
 3   icu_intime                                                       14289 non-null  int64  
 4   icu_outtime                                                      14289 non-null  int64  
 5   los_icu                                                          14289 non-null  float64
 6   icu_death                                                        14289 non-null  int64  
 7   gender                                  

In [ ]:
numerical_features = train_data.select_dtypes(include=['float64', 'int64']).columns.drop(['id', 'hosp_admittime', 'hosp_dischtime', 'icu_intime',  'icu_outtime', 'los_icu', 'time_to_icu_discharge', 'weight_admit', 'height'])
X_train = train_data[numerical_features]
y_train = train_data['time_to_icu_discharge']

X_test = test_data[numerical_features]
y_test = test_data['time_to_icu_discharge']

In [ ]:
numerical_features

Index(['icu_death', 'gender', 'admission_age', 'charlson_score',
       'atrial_fibrillation', 'malignant_cancer', 'chf', 'ckd', 'cld', 'copd',
       'diabetes', 'hypertension', 'ihd', 'stroke', 'icu_outcome',
       'race_encode_African', 'race_encode_Asian', 'race_encode_Caucasian',
       'race_encode_Hispanic', 'race_encode_Not Specified',
       'race_encode_South American', 'admission_type_DIRECT EMER.',
       'admission_type_DIRECT OBSERVATION', 'admission_type_ELECTIVE',
       'admission_type_EU OBSERVATION', 'admission_type_EW EMER.',
       'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT',
       'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
       'first_careunit_Coronary Care Unit (CCU)',
       'first_careunit_Medical Intensive Care Unit (MICU)',
       'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
       'first_careunit_Neuro Intermediate', 'first_careunit_Neuro Stepdo

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Reshape X_train_scaled and X_test_scaled for CNN input:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(50, activation='relu', kernel_regularizer=l2(0.001)),  # L2 regularization
    Dropout(0.5),
    Dense(1)
])

model.compile(optimizer=Adam(), loss='mse')

In [ ]:
model.fit(X_train_reshaped, y_train, epochs=100, verbose=1)

Epoch 1/100
447/447 [==============================] - 4s 7ms/step - loss: 697449404432384.0000
Epoch 2/100
447/447 [==============================] - 2s 5ms/step - loss: 697430144188416.0000
Epoch 3/100
447/447 [==============================] - 2s 5ms/step - loss: 697408333807616.0000
Epoch 4/100
447/447 [==============================] - 4s 9ms/step - loss: 697388670910464.0000
Epoch 5/100
447/447 [==============================] - 4s 9ms/step - loss: 697397327953920.0000
Epoch 6/100
447/447 [==============================] - 3s 7ms/step - loss: 697385651011584.0000
Epoch 7/100
447/447 [==============================] - 2s 4ms/step - loss: 697374645157888.0000
Epoch 8/100
447/447 [==============================] - 2s 4ms/step - loss: 697330755960832.0000
Epoch 9/100
447/447 [==============================] - 2s 4ms/step - loss: 697367531618304.0000
Epoch 10/100
447/447 [==============================] - 2s 4ms/step - loss: 697350754402304.0000
Epoch 11/100
447/447 [=================

In [ ]:
test_loss = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss}')

128/128 [==============================] - 0s 2ms/step - loss: 1933967011101016064.0000
Test Loss: 1.933967011101016e+18


In [ ]:
predictions = model.predict(X_test_reshaped)

128/128 [==============================] - 0s 2ms/step


In [ ]:
predictions

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)